https://github.com/AnswerDotAI/fsdp_qlora/blob/main/train.py (ref)

In [ ]:
# !pip install bitsandbytes
# !pip install hqq
# !pip install wandb

In [ ]:
import tqdm
from typing import List, Dict, Union
import torch
from torch import nn, Tensor
from bitsandbytes.nn import Linear4bit, Params4bit
import types

try:
    from hqq.core.quantize import HQQLinear, HQQBackend, BaseQuantizeConfig
except ImportError:
    HQQLinear = None
    pass

try:
    import wandb
except ImportError:
    pass

ModuleNotFoundError: No module named 'bitsandbytes'

In [ ]:
class Logger:
    def __init__(self, args, log_to='stdout', project_name='fsdp_qlora',
                 entity=None, group=None, name=None, rank=0):
        self.log_to = log_to
        if self.log_to == 'wandb' and rank==0:
            import wandb
            wandb.init(
                project=project_name,
                entity=entity,
                group=group,
                name=name,
                cofig=args
            )

    def log(self, d:Dict, rank:int):
        if rank != 0:
            return
        if self.log_to == "tqdm":
            for k,v in d.items():
                tqdm.write(f'{k}: {v}')
        elif self.log_to == 'wandb':
            wandb.log(d)
        elif self.log_to == 'stdout':
            for k,v in d.item():
                print(f'{k}: {v}')


In [ ]:
def update_progress_bar(progress_bar:tqdm, epoch:int, log_loss:float, log_lr:float, rank:int):
    if rank==0:
        if log_lr >= 0:
            progress_bar.set_description(f"epoch {epoch}, loss {log_loss:.3f}, lr {log_lr:.2e}", refresh=True)
        else:
            progress_bar.set_description(f"epoch {epoch}, loss {log_loss:.3f}", refresh=True)

In [ ]:
def replace_linear(model:nn.Module, linear_replacement:nn.Module, quant_config:Union[dict,None]=None,
                  skip_modules:List[str]=['lm_head'], **kwargs):
    for name, module in model.named_children():
        if len(list(module.children())) > 0:
            replace_linear(module, linear_replacement, quant_config, skip_modules, **kwargs)

        if isinstance(module, nn.Linear) and name not in skip_modules:
            if issubclass(linear_replacement, Linear4bit):
                model._modules[name] = linear_replacement(
                    module.in_features,
                    module.out_features,
                    module.bias is not None,
                    **kwargs
                )
            elif issubclass(linear_replacement, HQQLinear):
                model._modules[name] = linear_replacement(module, quant_config, **kwargs)
            else:
                raise ValueError(f"unsupported linear replacement: {type(linear_replacement)}")
    return model

In [ ]:
m = nn.Sequential(
    nn.Linear(5, 10),
    nn.Sequential(nn.Linear(10, 50), nn.ReLU())
)
m = m.to('cuda')
m

Sequential(
  (0): Linear(in_features=5, out_features=10, bias=True)
  (1): Sequential(
    (0): Linear(in_features=10, out_features=50, bias=True)
    (1): ReLU()
  )
)

In [ ]:
for p in m.parameters():
    print(p.shape, p.dtype, p.requires_grad, p.device)

torch.Size([10, 5]) torch.float32 True cuda:0
torch.Size([10]) torch.float32 True cuda:0
torch.Size([50, 10]) torch.float32 True cuda:0
torch.Size([50]) torch.float32 True cuda:0


In [ ]:
r = replace_linear(m, linear_replacement=Linear4bit).to('cuda')
r

Sequential(
  (0): Linear4bit(in_features=5, out_features=10, bias=True)
  (1): Sequential(
    (0): Linear4bit(in_features=10, out_features=50, bias=True)
    (1): ReLU()
  )
)

In [ ]:
for p in r.parameters():
    print(p.shape, p.dtype, p.requires_grad, p.device)

torch.Size([25, 1]) torch.uint8 False cuda:0
torch.Size([10]) torch.float32 True cuda:0
torch.Size([250, 1]) torch.uint8 False cuda:0
torch.Size([50]) torch.float32 True cuda:0


In [ ]:
def setup_quantized_meta_for_peft(model:nn.Module):
  def temp_to_method(self, *args, **kwargs):
    return self
  for param in model.parameters():
    if isinstance(param, Params4bit):
      param.quant_state._orig_to = param.quant_state.to
      param.quant_state.to = types.MethodType(temp_to_method, param.quant_state)

In [ ]:
def setup_quantized_peft_meta_for_training(model:nn.Module):
  for param in model.parameters():
    if isinstance(param, Params4bit) and hasattr(param.quant_state, '_orig_to'):
      param.quant_state.to = param.quant_state._orig_to
      param.quant_state._orig_to = None

In [ ]:
name = "nn.Conv2d"
module_key, _, value_key = name.rpartition('.')

print("module_key:", module_key)
print("value_key:", value_key)
print('_', _)

module_key: nn
value_key: Conv2d
_ .


In [ ]:
def load_and_quantize(module:nn.Module, name:str, value:Tensor, device:torch.device=None, dtype:torch.dtype=None,
                      skip_names:list[str]=[], is_meta_rank:bool=False, low_memory:bool=True, verbose:bool=False, quant_method:str='bnb'):

  def place_on_device(value):
    if is_meta_rank:
      device = 'meta'
    elif low_memory:
      device = 'cpu'
    return value.to(device=device)

    if any([skip_name in name for skip_name in skip_names]):
      if verbose:
        print(f'skipping {name} because it is in skip_names')
      return

    module_key, _, value_key = name.rpartition('.')
    try:
      submodule = module.get_submodule(module_key)
    except AttributeError as e:
      print(f'module {module_key} not found:\n{e}')
      return

    try:
      if quant_method=='bnb':
          param = submodule.get_parameter(value_key)
          if isinstance(param, Params4bit):
            value = type(param)(value.to(device=device, dtype=dtype).data, **param.__dict__).cuda(device)
            if is_meta_rank:
              value = type(param)(value.data.to('meta'), **value.__dict__)
            elif low_memory:
              value = type(param)(value.data.to("cpu"), **value.__dict__)
          else:
            value = type(param)(place_on_device(value).data)
      elif quant_method=='hqq':
        if isinstance(submodule, HQQLinear):
          if value_key == 'weight':
            submodule.linear_layer.to_empty(device=device)
            submodule.linear_layer.weight.data.copy_(value.to(device=device, dtype=dtype))
            submodule.initialize()

            if is_meta_rank:
              setattr(submodule, "W_q", nn.Parameter(submodule.W_q.to('meta')))
            elif low_memory:
              setattr(submodule, "W_q", nn.Parameter(submodule.W_q.to("cpu")))
            submodule.in_gpu = False

          if value_key == "bias":
            raise ValueError('bias not supported in HQQLinear yet')
        else:
          param = submodule.get_parameter(value_key)
          value = type(param)(place_on_device(value).data)

    except AttributeError:
      # remove pass from source code
      value = place_on_device(value)

    if HQQLinear is None or not isinstance(submodule, HQQLinear):
      setattr(submodule, value_key, value)



In [ ]:
PROMPT_DICT = {
    "prompt_input": (
        "below is an instruction that describes a task, paired with an input that provides further context. "
        "write a response that appropriately completes the request.\n\n"
        "### instruction:\n{instruction}\n\n### input:\n{input}\n\n### response:"
    ),
    "prompt_no_input": (
        "below is an instruction that describes a task. "
        "write a response that appropriately completes the request.\n\n"
        "### instrucion:\n{instruction}\n\n### response:"
    )
}

In [ ]:
class InstructionDataset(Dataset):
    def __init__(self, dataset, tokenizer, style="alpaca"):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.style = style

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        IGNORE_INDEX = -100  # The default setting in CrossEntropyLoss
        if self.style == "guanaco":
            prompt = self.dataset[index]["text"].split("### Assistant: ")[0]
            example = self.dataset[index]["text"]
        elif self.style == "qna":
            prompt_template = "###Context:\n{context}\n###Question:\n{question}\n###Answer:\n"
            sample = self.dataset[index]
            prompt = prompt_template.format_map(sample)
            example = prompt + sample['answer']
        else: # Alpaca
            ann = self.dataset[index]
            if ann.get("input", "") == "":
                prompt = PROMPT_DICT["prompt_no_input"].format_map(ann)
            else:
                prompt = PROMPT_DICT["prompt_input"].format_map(ann)
            example = prompt + ann["output"]

        prompt = torch.tensor(
            self.tokenizer.encode(prompt), dtype=torch.int64
        )
        example = self.tokenizer.encode(example)
        example.append(self.tokenizer.eos_token_id)
        example = torch.tensor(
            example, dtype=torch.int64
        )
        labels = copy.deepcopy(example)
        labels[: len(prompt)] = -1
        example_mask = example.ge(0)
        label_mask = labels.ge(0)
        example[~example_mask] = 0
        labels[~label_mask] = IGNORE_INDEX

        return {
            "input_ids": example.tolist(),
            "labels": labels.tolist(),
            "attention_mask":example_mask.tolist(),
        }